In [0]:
# !unzip -q "dataset.zip"

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
root = ',/'

import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models, datasets, transforms

from PIL import Image

%matplotlib inline

# Create Dataset

In [0]:
data_dir = f'{root}/dataset'
batch_size = 32

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(), # convert 1 to 3 channels
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Train dataset and dataloader
train_dataset = datasets.ImageFolder(f'{data_dir}/train', transform=transform)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

train_sizes = len(train_dataset)
class_names = train_dataset.classes

# Validation dataset and dataloader
val_dataset = datasets.ImageFolder(f'{data_dir}/val', transform)

val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)

val_sizes = len(val_dataset)

In [13]:
class_names

['fall', 'not_fall']

In [14]:
train_sizes, val_sizes

(11173, 3310)

In [15]:
len(train_dataset)

11173

# Create Model

In [0]:
class FDNet(nn.Module):
    def __init__(self, out_features=2):
        super(FDNet, self).__init__()
        mnet = models.mobilenet_v2(pretrained=True)
        for name, param in mnet.named_parameters():
            if("bn" not in name):
                param.requires_grad_(False)
            
        # Parameters of newly constructed modules have requires_grad=True by default
        in_features = mnet.classifier[1].in_features
        mnet.classifier = nn.Sequential(
                                nn.Dropout(p=0.2, inplace=False),
                                nn.Linear(in_features,500),
                                nn.ReLU(),
                                nn.Dropout(),
                                nn.Linear(500, out_features))
        self.mnet = mnet
        
    def forward(self, images):
        features = self.mnet(images)
        
        return features

## FDNet

In [17]:
# Initialize the model.
model = FDNet()

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/checkpoints/mobilenet_v2-b0353104.pth


FDNet(
  (mnet): MobileNetV2(
    (features): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

# Train Model

In [0]:
def train(model, optimizer, loss_fn, train_loader, val_loader, start_epoch = 0, epochs=20, device="cpu"):
    model.to(device)
    for epoch in range(start_epoch, start_epoch+epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [0]:
# Initialize the model.
model = FDNet()

# Define the loss function.
criterion = nn.CrossEntropyLoss()

# Define the optimizer.
params = model.mnet.classifier.parameters()
optimizer = optim.SGD(params, lr=0.001, momentum=0.9)

epochs = 20

In [20]:
%%time
train(model, optimizer, criterion, train_dataloader, val_dataloader, epochs=epochs, device=device)

Epoch: 0, Training Loss: 0.23, Validation Loss: 0.14, accuracy = 0.95
Epoch: 1, Training Loss: 0.16, Validation Loss: 0.12, accuracy = 0.96
Epoch: 2, Training Loss: 0.14, Validation Loss: 0.11, accuracy = 0.97
Epoch: 3, Training Loss: 0.13, Validation Loss: 0.10, accuracy = 0.97
Epoch: 4, Training Loss: 0.12, Validation Loss: 0.10, accuracy = 0.97
Epoch: 5, Training Loss: 0.11, Validation Loss: 0.09, accuracy = 0.97
Epoch: 6, Training Loss: 0.11, Validation Loss: 0.09, accuracy = 0.97
Epoch: 7, Training Loss: 0.10, Validation Loss: 0.09, accuracy = 0.97
Epoch: 8, Training Loss: 0.10, Validation Loss: 0.09, accuracy = 0.97
Epoch: 9, Training Loss: 0.10, Validation Loss: 0.09, accuracy = 0.97
Epoch: 10, Training Loss: 0.10, Validation Loss: 0.09, accuracy = 0.97
Epoch: 11, Training Loss: 0.10, Validation Loss: 0.09, accuracy = 0.97
Epoch: 12, Training Loss: 0.10, Validation Loss: 0.08, accuracy = 0.97
Epoch: 13, Training Loss: 0.09, Validation Loss: 0.08, accuracy = 0.97
Epoch: 14, Train

In [0]:
os.makedirs(f'{root}/train_model', exist_ok=True)
PATH = f'{root}/train_model/fdnet.pt'
torch.save(model.state_dict(), PATH)

# Evaluation

In [0]:
# Test dataset and dataloader
test_dataset = datasets.ImageFolder(f'{data_dir}/test', transform)

test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

test_sizes = len(test_dataset)

In [29]:
test_sizes

1458

In [30]:
outputs_np = np.empty((0), dtype=int)
targets_np = np.empty((0), dtype=int)

model.eval()
num_correct = 0 
num_examples = 0
for batch in test_dataloader:
    inputs, targets = batch
    inputs = inputs.to(device)
    outputs = model(inputs)
    outputs = torch.max(F.softmax(outputs, dim=1), dim=1)[1]
    targets = targets.to(device)
    correct = torch.eq(outputs, targets).view(-1)
    num_correct += torch.sum(correct).item()
    num_examples += correct.shape[0]
    
    outputs_np = np.concatenate([outputs_np, outputs.cpu().numpy()], axis=None)
    targets_np = np.concatenate([targets_np, targets.cpu().numpy()], axis=None)

print(f'Accuracy: {num_correct / num_examples:.2f}')
print(num_correct, num_examples)


Accuracy: 0.95
1378 1458


## Confusion Matrix

In [31]:
from sklearn.metrics import confusion_matrix

cf = confusion_matrix(targets_np, outputs_np)

cf

array([[  61,   59],
       [  21, 1317]])

In [32]:
tn, fp, fn, tp = cf.ravel()

tn, fp, fn, tp

(61, 59, 21, 1317)